In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from io import StringIO
from pathlib import Path

In [ ]:
season_code = "25_26"
current_dir = Path.cwd() 
path_folder = current_dir.parent.parent.parent / "csv" / f"csv{season_code}" / "raw data scouting"
path_folder_end = current_dir.parent.parent / "csv" / f"csv{season_code}" / "scouting"


/Users/matteolemesre/Desktop/Data Project/Data LOSC/csv/csv25_26/raw data scouting
/Users/matteolemesre/Desktop/Data Project/Data LOSC/Github/csv/csv25_26/scouting


# Scrapping

In [19]:
url_template = "https://fbref.com/en/comps/"
url_end       = "-Stats"

url_stats     = ["stats", "shooting", "passing", "passing_types", "gca", "defense", "possession", "misc"]
tables_id     = ["stats_standard", "stats_shooting", "stats_passing", "stats_passing_types", "stats_gca", "stats_defense", "stats_possession", "stats_misc"]
stats_button  = ["Standard Stats", "Shooting", "Passing", "Pass Types", "Goal and Shot Creation", "Defensive Actions", "Possession", "Miscellaneous Stats"]

url_stats_gk     = ["keepers", "keepersadv"]
tables_id_gk     = ["stats_keeper", "stats_keeper_adv"]
stats_button_gk  = ["Keeper", "Keeper Adv"]

others_leagues_infos = [
    ["Argentine Primera", "21", "Liga-Profesional-Argentina"],
    ["Brazilian Série A", "24", "Serie-A"],
    ["Dutch Eredivisie", "23", "Eredivisie"],
    ["Major League Soccer", "22", "Major-League-Soccer"],
    ["Portuguese Primeira Liga", "32", "Primeira-Liga"],
    ["Copa Libertadores", "14", "Copa-Libertadores"],
    ["English Championship", "10", "Championship"],
    ["Italian Serie B", "18", "Serie-B"],
    ["Mexican Liga MX", "31", "Liga-MX"],
    ["Belgian Pro League", "37", "Belgian-Pro-League"]
]

leagues_list = [others_leagues_infos]
leagues_list_names = ["Scouting"]

In [ ]:
def extract_table(driver, table_id):
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/17.0"
    )
    current_dir = Path.cwd()
    path_chrome = current_dir.parent.parent.parent.parent.parent / "chromedriver2" / "chromedriver"
    service = Service(path_chrome)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url_stats)
    time.sleep(random.uniform(5, 7))
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html, "html.parser")
    
    tables = soup.find_all("table")

    if tables:
        data = pd.read_html(StringIO(str(tables)))[0]
        return data.drop_duplicates()
    
    return None

In [21]:
def data_stat(url, table_id, stat_button):
    service = Service(executable_path="/Users/matteolemesre/Desktop/Data Project/chromedriver2/chromedriver")
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    
    driver = webdriver.Chrome(service=service, options=options)

    try:
        driver.get(url)
        time.sleep(5)
        df = extract_table(driver, table_id)
        
        if df is None:
            try:
                button = driver.find_element(By.XPATH, f"//button[contains(text(), 'Show Player {stat_button}')]")
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", button)
                time.sleep(2)

                df = extract_table(driver, table_id)
            except Exception as e:
                df = None
    finally:
        driver.quit()

    return df

In [22]:
def data_stats(league_infos, tables_id, url_stats, url_template, url_end, stats_button):
    df_league = pd.DataFrame()
    
    for i, stat in enumerate(url_stats):
        url = url_template + league_infos[1] + "/" + stat + "/" + league_infos[2] + url_end
        table_id = tables_id[i]
        stat_button = stats_button[i]
        df_stats = data_stat(url, table_id, stat_button)
        df_league = pd.concat([df_league, df_stats], axis=1)
        
    return df_league

In [23]:
def data_stats_leagues(leagues_list_name, leagues_infos, tables_id, url_stats, url_template, url_end, stats_button, suff):
    
    for league_infos in leagues_infos:
        print(f" - {league_infos[0]}")
        df_league = data_stats(league_infos, tables_id, url_stats, url_template, url_end, stats_button)
        file_end_league = path_folder + leagues_list_name + "/" + league_infos[0] + suff + ".csv"
        df_league.to_csv(file_end_league, index=False)
        if (suff == "" and df_league.shape[1] == 225) or (suff == "_gk" and df_league.shape[1] == 61):
            print(f"  --> Validated") 
        else:
            print(f"  --> Refused — got {df_league.shape[1]} columns")
        
    return 

In [24]:
suff = ""
data_stats_leagues(leagues_list_names[0], leagues_list[0], tables_id, url_stats, url_template, url_end, stats_button, suff)
    
suff = "_gk"
data_stats_leagues(leagues_list_names[0], leagues_list[0], tables_id_gk, url_stats_gk, url_template, url_end, stats_button_gk, suff)

For Scouting:
 - Argentine Primera
  --> Validated
 - Brazilian Série A
  --> Validated
 - Dutch Eredivisie
  --> Validated
 - Major League Soccer
  --> Validated
 - Portuguese Primeira Liga
  --> Validated
 - Copa Libertadores
  --> Validated
 - English Championship
  --> Validated
 - Italian Serie B
  --> Validated
 - Mexican Liga MX
  --> Validated
 - Belgian Pro League
  --> Validated
 - Argentine Primera
  --> Validated
 - Brazilian Série A
  --> Validated
 - Dutch Eredivisie
  --> Validated
 - Major League Soccer
  --> Validated
 - Portuguese Primeira Liga
  --> Validated
 - Copa Libertadores
  --> Validated
 - English Championship
  --> Validated
 - Italian Serie B
  --> Validated
 - Mexican Liga MX
  --> Validated
 - Belgian Pro League
  --> Validated


In [32]:
for i, league_infos in enumerate(leagues_list_names):
    df_leagues = pd.DataFrame()
    df_leagues_gk = pd.DataFrame()
    
    path = path_folder + f"{leagues_list_names[i]}/"
    
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            file_path = os.path.join(path, filename)
            df = pd.read_csv(file_path)

            if filename.endswith("Copa Libertadores.csv") or filename.endswith("Copa Libertadores_gk.csv"):
                df['Unnamed: 4_level_0'] = df['Unnamed: 4_level_0'].astype(str).str.replace(r'^[a-z]{2,3}\s+', '', regex=True)
                
            if filename.endswith("_gk.csv"):
                df_leagues_gk = pd.concat([df_leagues_gk, df], axis=0)
            else:
                df_leagues = pd.concat([df_leagues, df], axis=0)
                
    df_leagues.to_csv(os.path.join(path_folder, f"{leagues_list_names[i]}.csv"), index=False)
    df_leagues_gk.to_csv(os.path.join(path_folder, f"{leagues_list_names[i]}_gk.csv"), index=False)

# Cleaning

In [33]:
def clean_age(age_str):
    if isinstance(age_str, str) and "-" in age_str:
        try:
            return int(age_str.split("-")[0])
        except ValueError:
            return None
    elif isinstance(age_str, (int, float)):
        try:
            return int(age_str)
        except ValueError:
            return None
    return None

In [34]:
def new_position(position_raw):
    if position_raw in ["FW", "FW,MF", "MF,FW", "FW,DF", "DF,FW"]:
        return "Forward"
    elif position_raw in ["MF", "MF,DF", "DF,MF"]:
        return "Midfielder"
    elif position_raw in ["DF"]:
        return "Defender"
    elif position_raw in ["GK"]:
        return "Goalkeeper"
    else:
        return ""


In [35]:
def clean_nationality(nat):
    try:
        return str(nat).split()[-1].upper()
    except:
        return np.nan

In [36]:
def extract_and_rename_stats(df):
    df.columns = [
        f"{top.strip()} - {bottom.strip()}" if "Unnamed" not in top else bottom.strip()
        for top, bottom in df.columns
    ]
    df.columns = df.columns.astype(str).str.strip()
    df.columns.name = None

    columns_rename_map = {
        "Player": "Player",
        "Nation": "Nation",
        "Pos": "General Position",
        'Squad': 'Team',
        "Age": "Age",
        "Born": "Born",
        
        'Playing Time - MP': 'Matches Played',
        'Playing Time.1 - Starts': 'Starts',
        'Playing Time.2 - Min': 'Minutes Played',
        
        "Performance - Gls": "Goals",
        "Expected - xG": "xG",
        "Standard.1 - Sh": "Shots",
        "Standard.2 - SoT": "Shots on Target",
        "Standard.10 - PK": "Penalties Scored",
        "Standard.11 - PKatt": "Penalties Attempted",
        "SCA - SCA": "Shot Creating Actions",
        "GCA - GCA": "Goal Creating Actions",
        
        "Total - Cmp": "Passes Completed (Total)",
        "Total.1 - Att": "Passes Attempted (Total)",
        "Short - Cmp": "Passes Completed (Short)",
        "Short.1 - Att": "Passes Attempted (Short)",
        "Medium - Cmp": "Passes Completed (Medium)",
        "Medium.1 - Att": "Passes Attempted (Medium)",
        "Long - Cmp": "Passes Completed (Long)",
        "Long.1 - Att": "Passes Attempted (Long)",
        
        "Performance.1 - Ast": "Assists",
        "Expected - xAG": "Expected Assists (xAG)",
        "Expected.9 - xA": "Expected Assists (xA)",
        
        "KP": "Key Passes",
        "1/3": "Passes into Final Third",
        "PPA": "Passes into Penalty Area",
        "CrsPA": "Crosses into Penalty Area",
        "PrgP": "Progressive Passes",
        
        "Carries.3 - PrgC": "Progressive Carries",
        "Receiving.1 - PrgR": "Progressive Runs",
        
        "Tackles - Tkl": "Tackles",
        "Tackles.1 - TklW": "Tackles Won",
        "Tackles.2 - Def 3rd": "Tackles Defensive Third",
        "Tackles.3 - Mid 3rd": "Tackles Middle Third",
        "Tackles.4 - Att 3rd": "Tackles Attacking Third",
        
        "Challenges - Tkl": "Challenges Tackled",
        "Challenges.1 - Att": "Challenges Attempted",
        "Challenges.3 - Lost": "Challenges Lost",
        
        "Blocks - Blocks": "Blocks",
        
        "Int": "Interceptions",
        "Clr": "Clearances",
        "Err": "Errors",
        
        "Touches - Touches": "Touches",
        "Touches.1 - Def Pen": "Touches Defensive Penalty Area",
        "Touches.2 - Def 3rd": "Touches Defensive Third",
        "Touches.3 - Mid 3rd": "Touches Middle Third",
        "Touches.4 - Att 3rd": "Touches Attacking Third",
        "Touches.5 - Att Pen": "Touches Attacking Penalty Area",
        "Touches.6 - Live": "Live-Ball Touches",
        
        "Take-Ons - Att": "Take-Ons Attempted",
        "Take-Ons.1 - Succ": "Successful Take-Ons",
        
        "Carries - Carries": "Carries",
        "Carries.3 - PrgC": "Progressive Carries",
        "Carries.4 - 1/3": "Carries into Final Third",
        "Carries.5 - CPA": "Carries into Penalty Area",
        "Carries.6 - Mis": "Miscontrols",
        "Carries.7 - Dis": "Dispossessed",
        
        "Receiving - Rec": "Passes Received",
        "Receiving.1 - PrgR": "Progressive Passes Received",
        
        "Performance.8 - CrdY": "Yellow Cards",
        "Performance.9 - CrdR": "Red Cards",
        "Performance.10 - 2CrdY": "Second Yellow Cards",
        "Performance.11 - Fls": "Fouls Committed",
        "Performance.12 - Fld": "Fouls Drawn",
        "Performance.13 - Off": "Offsides",
        "Performance.17 - PKwon": "Penalties Won",
        "Performance.18 - PKcon": "Penalties Conceded",
        "Performance.19 - OG": "Own Goals",
        "Performance.20 - Recov": "Ball Recoveries",
        
        "Aerial Duels - Won": "Aerial Duels Won",
        "Aerial Duels.1 - Lost": "Aerial Duels Lost"
    }

    valid_columns = [col for col in columns_rename_map.keys() if col in df.columns]
    df_selected = df[valid_columns].copy()

    df_selected.rename(columns=columns_rename_map, inplace=True)

    df_selected.replace([np.inf, -np.inf], 0, inplace=True)
    df_selected.fillna(0, inplace=True)

    for col in df_selected.select_dtypes(include=['float64', 'int64']).columns:
        df_selected[col] = df_selected[col].round(2)
    
    df_selected = df_selected.loc[:, ~df_selected.columns.duplicated()]
    df_selected["Age"] = df_selected["Age"].apply(clean_age).astype("Int64")
    df_selected['General Position'] = df_selected['General Position'].astype(str).apply(new_position)
    df_selected["Nation"] = df_selected["Nation"].apply(clean_nationality)
    df_selected = df_selected[df_selected['General Position'] != 'Pos']
    df_selected = df_selected[df_selected['Player'] != 'Player']

    return df_selected

In [37]:
def extract_and_rename_stats_goals(df):
    df.columns = [
        f"{top.strip()} - {bottom.strip()}" if "Unnamed" not in top else bottom.strip()
        for top, bottom in df.columns
    ]
    df.columns = df.columns.astype(str).str.strip()
    df.columns.name = None
    
    columns_rename_map = {
        'Player': 'Player',
        'Nation': 'Nation',
        'Pos': 'General Position',
        'Squad': 'Team',
        'Age': 'Age',
        'Born': 'Born',

        'Playing Time - MP': 'Matches Played',
        'Playing Time.1 - Starts': 'Starts',
        'Playing Time.2 - Min': 'Minutes Played',

        'Performance - GA': 'Goals Against',
        'Performance.2 - SoTA': 'Shots on Target Against',
        'Performance.3 - Saves': 'Saves',
        'Performance.8 - CS': 'Clean Sheets',

        'Penalty Kicks - PKatt': 'Penalty Kicks Attempted',
        'Penalty Kicks.1 - PKA': 'Penalty Kicks Allowed',
        'Penalty Kicks.2 - PKsv': 'Penalty Kicks Saved',
        'Penalty Kicks.3 - PKm': 'Penalty Kicks Missed',

        'Expected - PSxG': 'Post-Shot Expected Goals',

        'Launched - Cmp': 'Launched Passes Completed',
        'Launched.1 - Att': 'Launched Passes Attempted',

        'Passes - Att (GK)': 'Passes Attempted (GK)',
        'Passes.1 - Thr': 'Through Balls',

        'Goal Kicks - Att': 'Goal Kicks Attempted',

        'Crosses - Opp': 'Crosses Opposed',
        'Crosses.1 - Stp': 'Crosses Stopped',

        'Sweeper - #OPA': 'Defensive Actions Outside Penalty Area',
    }

    valid_columns = [col for col in columns_rename_map if col in df.columns]
    df_selected = df[valid_columns].copy()

    df_selected.rename(columns=columns_rename_map, inplace=True)

    df_selected.replace([np.inf, -np.inf], 0, inplace=True)
    df_selected.fillna(0, inplace=True)
    
    for col in df_selected.select_dtypes(include=['float64', 'int64']).columns:
        df_selected[col] = df_selected[col].round(2)
    
    df_selected = df_selected.loc[:, ~df_selected.columns.duplicated()]
    df_selected["Age"] = df_selected["Age"].apply(clean_age).astype("Int64")
    df_selected['General Position'] = df_selected['General Position'].astype(str).apply(new_position)
    df_selected["Nation"] = df_selected["Nation"].apply(clean_nationality)
    df_selected = df_selected[df_selected['General Position'] != 'Pos']
    df_selected = df_selected[df_selected['Player'] != 'Player']
    
    return df_selected

In [38]:
leagues_list = [others_leagues_infos]
leagues_list_names = ["Scouting"]
                
for i, league_infos in enumerate(leagues_list):
    path = path_folder + leagues_list_names[i] + ".csv"
    df = pd.read_csv(path, header=[0, 1], low_memory=False)
    df = extract_and_rename_stats(df)
    path_end = path_folder_end + leagues_list_names[i] + ".csv"
    df.to_csv(path_end, index=False)
    
    path_goals = path_folder + leagues_list_names[i] + "_gk.csv"
    df_goals = pd.read_csv(path_goals, header=[0, 1], low_memory=False)
    df_goals = extract_and_rename_stats_goals(df_goals)
    path_end_goals = path_folder_end + leagues_list_names[i] + "_gk.csv"
    df_goals.to_csv(path_end_goals, index=False)